# COPT and LOLP Calculator

## import and settting printer

In [1]:
import itertools

import numpy as np
import pandas as pd

In [2]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

## inputs

### choose between using csv file or manually edit the notebook

In [3]:
# NOTE:
#   UNCOMMENT AND USE THIS CELL IF USING CSV INPUT
#   COMMENT IF USING MANUAL INPUT
# csv input path
df = pd.read_csv('../data/COPT Case.csv')

# extract csv
capacities = df['Capacity (MW)'].tolist()
outage_rates = df['FOR (Outage)'].tolist()
status = [True] * len(df.index)

In [4]:
# # NOTE:
# #   UNCOMMENT AND USE THIS CELL IF USING MANUAL INPUT
# #   COMMENT IF USING CSV INPUT
# # max capacity
# capacities = [
#     80,
#     70,
#     40,
#     50,
#     40,
#     50
# ]

# # forced outage rate
# outage_rates = [
#     0.1, 
#     0.2,
#     0.1,
#     0.2,
#     0.1,
#     0.1
# ]


# # status of generator,
# #     True: exist
# #     False: not exist
# status = [
#     True,
#     False,
#     True,
#     True,
#     True,
#     True,
# ]

In [5]:
pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

,capacities,outage_rates,status
1,50,0.100,True
2,50,0.100,True
3,50,0.100,True
4,30,0.100,True
5,10,0.070,True
6,20,0.070,True
7,10,0.100,True
8,5,0.100,True
9,10,0.070,True
10,10,0.070,True


In [6]:
# peak load or load at time considering
# used to calculate LOLP
demands = [590, 470, 585, 650, 660]

## Parameters

In [7]:
# NOTE:
#   There is two batch, bathc in evaluating of permutation and batch in making COPT.

BATCH_PRODUCT = 100  # batch size for each making table before combine duplicate
MIN_COVERAGE = 1  # early stopper
BATCH_GENERATOR = 10

# TODO:
#   1. Implement resample, useful for big COPT table, triggered only if max table length achieved

## COPT table

In [8]:
generator_list = [[cap, 1-out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]

tables = [np.array([generator, [0, 1 - generator[1]]]) for generator in generator_list]

table = tables[0].copy()  # copy to avoid modifying data
for table_ in tables[1:]:
    # faster than flatten + transpose
    table = np.hstack(((table[:, 0, None] + table_[:, 0]).reshape(-1,1),
                       (table[:, 1, None] * table_[:, 1]).reshape(-1,1)))

    # sort table
    table = table[(-table[:, 0]).argsort(),:]

    # combine duplicate
    table = np.array([[k,sum([x[1] for x in list(g)])] 
                    for k,g in itertools.groupby(table, lambda x:x[0])])

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

df = pd.DataFrame(data=table,
                  columns=columns_name,
                  index=pd.RangeIndex(1, len(table) + 1, 1))
df.to_csv('../results/merged.csv')
df.head(10)

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
1,875.000,0.024,1.000,0.024
2,870.000,0.020,0.976,0.045
3,865.000,0.042,0.955,0.086
4,860.000,0.030,0.914,0.117
5,855.000,0.051,0.883,0.168
6,850.000,0.036,0.832,0.204
7,845.000,0.050,0.796,0.254
8,840.000,0.034,0.746,0.287
9,835.000,0.040,0.713,0.328
10,830.000,0.026,0.672,0.354


## LOLP

In [9]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

In [10]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
lolp

[0.00019334443587659824,
 1.4799527806098814e-07,
 0.0001449366283941029,
 0.003289599656014925,
 0.0050267079116176625]

In [11]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 0.0087
